In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import *

In [ ]:
csv = 's3a://t800-s3/dataset_t800/proyecto04/articles1.csv'
df1 = spark.read.csv(csv,inferSchema=True,header=True)
csv = 's3a://t800-s3/dataset_t800/proyecto04/articles2.csv'
df2 = spark.read.csv(csv,inferSchema=True,header=True)
csv = 's3a://t800-s3/dataset_t800/proyecto04/articles3.csv'
df3 = spark.read.csv(csv,inferSchema=True,header=True)

In [ ]:
join_1_df_2 = df1.join(df2, on=['_c0', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content'], how='left_outer')
full_df = join_1_df_2.join(df3, on=['_c0', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content'], how='left_outer')
full_df = full_df.limit(100)
full_df.show()

In [ ]:
reg = '[^a-zA-Z ]'
reg1 = '[\s*]{1,}'

In [ ]:
full_copy = full_df.withColumn("ltrimmed_word",ltrim(col("Content")))

full_df = full_copy.withColumn("clean", regexp_replace('ltrimmed_word', reg ,""))
full_df.select('ltrimmed_word','clean').show()

In [ ]:
full_df = full_df.withColumn("clean1", regexp_replace('clean', reg1 ," "))
full_df.select('clean','clean1').show(50)

In [ ]:
tokenization=Tokenizer(inputCol='clean1',outputCol='tokens')

In [ ]:
tokenized_df = tokenization.transform(full_df)

In [ ]:
tokenized_df.select('clean1','tokens').show(50)

In [ ]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [ ]:
refined_df=stopword_removal.transform(tokenized_df)

In [ ]:
refined_df.select('clean1','tokens','refined_tokens').show(50)

In [ ]:
refined_df

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.types import IntegerType

In [ ]:
len_udf = udf(lambda s: len(s), IntegerType())
refined_df = refined_df.withColumn("token_count", len_udf(col('refined_tokens')))

In [ ]:
refined_df.select('clean1','tokens','refined_tokens','token_count').show(50)

In [ ]:
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.feature import HashingTF, IDF

In [ ]:
aux_df = refined_df
aux_df = aux_df.drop('publication', 'author', 'publication', 'content', 'date', 'year', 'month', 'url', 'clean', 'clean1', 'tokens')
aux_df.show(50)

In [ ]:
fill = array().cast("array<string>")
tokens_a = when(col("refined_tokens").isNull(), fill).otherwise(col("refined_tokens"))
aux_df = aux_df.withColumn("refined_tokens", tokens_a)

In [ ]:
cv = CountVectorizer(inputCol="refined_tokens", outputCol="rawFeatures" )
cvmodel = cv.fit(aux_df)

In [ ]:
featurizedData = cvmodel.transform(aux_df)

In [ ]:
vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)

In [ ]:
idf = IDF()
# idf.setParams(inputCol="rawFeatures", outputCol="features")
# IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
# rescaledData = idfModel.transform(featurizedData)
# rescaledData.show(100)

In [ ]:
vocab_broadcast.value